In [93]:
import numpy as np
from collections import defaultdict

wordToSupertag = dict()
sufToSupertag = defaultdict(list)

with open('../../NLP_Resources/supertags.txt') as f:
    temp = [tuple(x.split()) for x in f]
    wordToSupertag = {word: tag for (word, tag) in temp}
    for word,tag in temp:
        sufToSupertag[word[-3:]].append(tag)

In [94]:
vowels = {'a', 'e', 'i', 'o', 'u', 'ę', 'ą', 'ó', 'y'}

def getRhyme(s):
    count = 0
    for i in range(len(s)-1, -1, -1):
        if s[i] == 'i' and i+1 < len(s) and s[i+1] in vowels:
            continue
        if s[i] in vowels:
            count += 1
        if count == 2:
            return s[i:]
    return s

def countSyllables(s):
    count = 0

    for i in range(len(s)):
        if s[i-1] == 'i' and s[i] in vowels:
            continue

        if s[i] in vowels:
            count += 1
    return count
        


In [95]:
wordCount = defaultdict(int)

with open('../../NLP_Resources/pan-tadeusz.txt') as f:
    dwuwers = []
    count = 0
    for line in f:
        line = line.strip().lower()
        if len(line.strip()) == 0:
            continue

        count +=1

        temp = []
        for word in line.split():
            t = ''

            for a in word:
                if a.isalpha():
                    t += a
            word = t

            wordCount[word] += 1 


In [96]:
import random

findRhymeDict = defaultdict(lambda: defaultdict(lambda: defaultdict(list))) #sylaby->tag->rym->[lista]
rhymeDict = defaultdict(lambda: defaultdict(lambda: defaultdict(set))) #rhyme->sylaby->tag->[lista]
syllablesDict = defaultdict(lambda: defaultdict(set)) #sylaba->tag->[lista]
dwuwersy = []

with open('../../NLP_Resources/pan-tadeusz.txt') as f:
    dwuwers = []
    count = 0
    for line in f:
        line = line.strip().lower()
        if len(line.strip()) == 0:
            continue

        count +=1

        temp = []
        for word in line.split():
            t = ''

            for a in word:
                if a.isalpha():
                    t += a

            word = t

            sylaby = countSyllables(word)

            if word in wordToSupertag:
                tag = wordToSupertag[word]
            else:
                tag = random.choice(sufToSupertag.get(word[-3:], ['grrrr']))
            temp.append((tag, sylaby))

            rym = getRhyme(word)

            findRhymeDict[sylaby][tag][rym].append(word)
            rhymeDict[rym][sylaby][tag].add(word)
            syllablesDict[sylaby][tag].add((word, wordCount[word]))

        dwuwers.append(temp)

        if count == 2:
            dwuwersy.append(dwuwers)
            dwuwers = []
            count = 0


In [97]:
def generatePT(numberOfLines):

    for i in range(numberOfLines):
        wers1, wers2 = ('', '')
        rym = ''
        while rym == '':
            wers1, wers2 = random.choice(dwuwersy)
            tag1 = wers1[-1][0] 
            tag2 = wers2[-1][0]
            sylaby1 = wers1[-1][1]
            sylaby2 = wers2[-1][1]
            keys = list(findRhymeDict[sylaby1][tag1].keys())
            random.shuffle(keys)
            for key in keys:
                if key in findRhymeDict[sylaby2][tag2]:
                    rym = key
                    break
        rym1 = random.choice(list(rhymeDict[rym][sylaby1][tag1]))

        if len(rhymeDict[rym][sylaby2][tag2]) > 1:
            rym2 = rym1
            while rym2 == rym1:
                rym2 = random.choice(list(rhymeDict[rym][sylaby2][tag2]))
        else:
            rym2 = random.choice(list(rhymeDict[rym][sylaby2][tag2]))

        def getLine(wers, ending):
            result = ''
            for i in range(len(wers)-1):
                tag, sylaby = wers[i]
                probs = np.array([x[1] for x in syllablesDict[sylaby][tag]])
                elements = np.array([x[0] for x in syllablesDict[sylaby][tag]])
                sum = np.sum(probs)
                word = np.random.choice(elements, p = probs/sum)
                result += word + ' '
            result += ending
            return result
        
        result1 = getLine(wers1, rym1)
        result2 = getLine(wers2, rym2)

        print(result1, result2, sep = '\n', end = '\n------------------------\n')



In [102]:
generatePT(10)

wojski przypomnij głowę na jutro na błonie
jutro i cesarz z wstecznym ryczeniem tak zionie
------------------------
i znudziło kogutki do każdym zabiciu
one zwykły do ryków szukać o powiciu
------------------------
tymczasem pytaniami dozoru zwodzono
nic nie ubił do tych zbóż wtem żydzisko bono
------------------------
jeszcze dążył spał jeszcze zostać gdy nań wsiadły
naprzykrzona brejgela i łoskot zajadły
------------------------
tak odgadywał stronę zielonej podpory
choć powszechną i wstrząsnął otrząsać niej zmory
------------------------
było godne zabójstwo niedźwiedź gospodarski
siekł się wkoło na wojnie jarzyny i dziarski
------------------------
bawił się między różne i podobne stoła
które mocą oparte wszczynały dokoła
------------------------
więc nic nie szepcąc bardzo na sobie stawili
zjechali po prus izby piankami zwątpili
------------------------
jechać to cieszyć hrabio daj wódkę siwuchy
sędzia polubi niechaj siedzą szpiegi zuchy
------------------------
i szumami ją dawno

In [103]:

bigramDict = defaultdict(int)

with open('../../NLP_Resources/pan-tadeusz.txt') as f:
    dwuwers = []
    count = 0
    for line in f:
        line = line.strip().lower()
        if len(line.strip()) == 0:
            continue

        count +=1

        temp = []
        for i in range(len(line.split()) - 1):
            word1,word2 = line.split()[i:i+2]

            t = ''
            for a in word1:
                if a.isalpha():
                    t += a
            word1 = t

            t = ''
            for a in word2:
                if a.isalpha():
                    t += a
            word2 = t

            bigramDict[word1+"#"+word2] += 1

In [149]:
def generatePTImproved(numberOfLines):

    for i in range(numberOfLines):
        rym1,rym2  = ('', '')
        while rym1 == rym2:
            wers1, wers2 = ('', '')
            rym = ''
            while rym == '':
                wers1, wers2 = random.choice(dwuwersy)
                tag1 = wers1[-1][0] 
                tag2 = wers2[-1][0]
                sylaby1 = wers1[-1][1]
                sylaby2 = wers2[-1][1]
                keys = list(findRhymeDict[sylaby1][tag1].keys())
                random.shuffle(keys)
                for key in keys:
                    if key in findRhymeDict[sylaby2][tag2]:
                        rym = key
                        break

            rym1 = random.choice(list(rhymeDict[rym][sylaby1][tag1]))

            if len(rhymeDict[rym][sylaby2][tag2]) > 1:
                rym2 = rym1
                while rym2 == rym1:
                    rym2 = random.choice(list(rhymeDict[rym][sylaby2][tag2]))
            else:
                rym2 = random.choice(list(rhymeDict[rym][sylaby2][tag2]))

        

        def getLine(wers, ending):
            result = ''
            prev = ending
            res = []
            wers.reverse()
            for i in range(1, len(wers)):
                tag, sylaby = wers[i]
                elements = [x[0] for x in syllablesDict[sylaby][tag]]
                temp = []
                for kandydat in syllablesDict[sylaby][tag]:
                    temp.append((bigramDict[kandydat[0]+"#"+prev]**2, kandydat[0]))
                temp = sorted(temp, reverse=True)
                #print(prev, temp)
                if temp[0][0] == 0:
                    probs = np.array([x[1] for x in syllablesDict[sylaby][tag]])
                    sum = np.sum(probs)
                    word = np.random.choice(elements, p = probs/sum)
                else:
                    word = temp[0][1]
                prev = word

                res.append(word)

            res.reverse()
            for i in range(len(res)):
                result += res[i] + ' '
            result += ending
            return result
        
        result1 = getLine(wers1, rym1)
        result2 = getLine(wers2, rym2)

        print(result1, result2, sep = '\n', end = '\n------------------------\n')

In [156]:
generatePTImproved(10)

i nazajutrz armija stóp rumaków zdrowia
słychać coraz to częściej jęk psiego wezgłowia
------------------------
żeby do swej ojczyzny nic więcej nie wiecie
więc po coście tu przyszli i czego wy chcecie
------------------------
a jako starzec często łysą głowę wchodzi
na ciele chorującym nim czuciem cwał godzi
------------------------
oprócz żabki na chorze tym kapela stała
i w klucznik i w czyśćcowe obyczaje brała
------------------------
słup już wiedli na zachód potem austryjacy
w szpilbergu zakopali mnie w strugach do pracy
------------------------
dość mnie nudzi ta cała historyja babia
a potem czegóż jeszcze wlazł mi tu ten hrabia
------------------------
jako maciek kiedym się ogromem wychował
wdział szlafmycę a ręce u ludzi zmarnował
------------------------
nie wydając nazwiska nie tylko na sznurkiem
ale nawet przed tobą i nad ruskim nurkiem
------------------------
czuła prawda że znowu podkomorzy trzymał
wiadomo że nikt z równym milczkiem zbyt się wstrzymał
------------------